## Introduction

This Notebook poses and answers some probability problems related to geometry. This is in keeping with
our theme of translating ideas and language into expressions in mathematics. 

By the way: As we make our way along to one of the solutions (to problem 3)
I hope to introduce you to some calculus... but it won't be too dreadful (I also hope).

### Sticks and Triangles


Suppose we come up with a way of identifying three sticks with particular lengths $a$, $b$, and $c$. 
Let's consider the question 'Can these three sticks be used to make a triangle?' Think for a moment about what
this means... can you write a mathematical requirement that is necessary to be able to make a triangle? 
(By *making a triangle* we mean the sticks touch end to end; there is no extra sticking out!)


Here is one way to write the mathematical condition needed for a triangle: 


Suppose $a$ is the longest of the three sticks. Of course $b$ and $c$ may have the same length as $a$ so we
can write $a \ge b$ and $a \ge c$. Then for a triangle to be possible we require $b+c>a$.


Let's call 'the probability that three sticks can make a triange' $P_\Delta$ for short. 


Now let's invent some stick rules with some randomness built in. The first two are from Martin Gardner's 
book **The Colossal Book Of Mathematics**. The third problem I may have seen somewhere but I forget where.
Here are all three problems so that you can think about them without having to go down further on this
page to find them.

1. Suppose you have a stick of length $1$ broken into three pieces: Two points are chosen at random
along the stick where it is cut to give three sticks. What is $P_\Delta$?

2. Suppose you have a stick of length $1$ broken into three pieces: First the stick is broken
at a randomly chosen point to produce two sticks. One of these sticks is chosen at random (50-50 chance)
and that stick is broken at a random point, resulting in three sticks in total. What is $P_\Delta$?

3. Suppose three sticks are produced each with random length between 0 and 1. What is $P_\Delta$?

Like other problems in probability we can resort to Monte Carlo methods in Python to answer these questions.
This has the advantage of avoiding solving the problem... or verifying an answer if we do come up with one.
The next three cells produce stastical answers for these three versions of the $P_\Delta$ problem, respectively.

In [23]:
################
##
## Three sticks = a triangle? Version 1: Choose two break points on the unit stick
##
number_of_trials = 100000
##
################

import random as r

number_of_triangles = 0

def Is_it_a_triangle(a, b, c):
    if a >= b and a >= c and b + c > a: return 1
    if b >= a and b >= c and a + c > b: return 1
    if c >= a and c >= b and a + b > c: return 1
    return 0

for trial in range(number_of_trials):
    
    # create the three sticks as random numbers between 0 and 1
    point_one = r.uniform(0.0, 1.0)
    point_two = r.uniform(0.0, 1.0)

    # order point_one and point_two so point_one is less than point_two
    if point_one > point_two:
        a = point_two
        b = point_one - point_two
        c = 1.0 - point_one
    else:
        a = point_one
        b = point_two - point_one
        c = 1.0 - point_two
    
    number_of_triangles += Is_it_a_triangle(a, b, c)

Pt = float(number_of_triangles) / float(number_of_trials)

print 'P(triangle) =', Pt, 'in', number_of_trials, 'tries'
    

P(triangle) = 0.25088 in 100000 tries


In [22]:
################
##
## Three sticks = a triangle? Version 2: Break 1.0 into two sticks, then break one of those sticks
##
number_of_trials = 100000
##
################

import random as r

number_of_triangles = 0

def Is_it_a_triangle(a, b, c):
    if a >= b and a >= c and b + c > a: return 1
    if b >= a and b >= c and a + c > b: return 1
    if c >= a and c >= b and a + b > c: return 1
    return 0

for trial in range(number_of_trials):

    # Break the original length-1 stick into two sticks
    first_stick = r.uniform(0.0, 1.0)
    second_stick = 1.0 - first_stick

    # We choose stick 1 or stick 2 to break
    if r.randint(1,2) == 1:                   # chooses to break the first stick
        if first_stick > second_stick:        #   which can make a triangle only if first > second
            
            first_stick_break = r.uniform(0.0, first_stick)
            a = first_stick_break
            b = first_stick - first_stick_break
            c = second_stick

            number_of_triangles += Is_it_a_triangle(a, b, c)
                
    else:                                      # we chose to break the second stick
        if first_stick < second_stick:         #   which can make a triangle only if second > first
            
            # Swap the labels on the two sticks
            temp = first_stick                 # This makes the part that follows identical to the above
            first_stick = second_stick    
            second_stick = temp                
            
            first_stick_break = r.uniform(0.0, first_stick)
            a = first_stick_break
            b = first_stick - first_stick_break
            c = second_stick

            number_of_triangles += Is_it_a_triangle(a, b, c)

Pt = float(number_of_triangles) / float(number_of_trials)

print 'P(triangle) =', Pt, 'in', number_of_trials, 'tries'
    

P(triangle) = 0.19269 in 1000000 tries


In [7]:
################
##
## Three sticks = a triangle? Version 3: 3 sticks each of random length from 0 to 1
##
number_of_trials = 1000000
##
################

import random as r

number_of_triangles = 0

def Is_it_a_triangle(a, b, c):
    if a >= b and a >= c and b + c > a: return 1
    if b >= a and b >= c and a + c > b: return 1
    if c >= a and c >= b and a + b > c: return 1
    return 0

for trial in range(number_of_trials):
    
    # create the three sticks as random numbers between 0 and 1
    a = r.uniform(0.0, 1.0)
    b = r.uniform(0.0, 1.0)
    c = r.uniform(0.0, 1.0)
    
    number_of_triangles += Is_it_a_triangle(a, b, c)

Pt = float(number_of_triangles) / float(number_of_trials)

print 'P(triangle) =', Pt, 'in', number_of_trials, 'tries'
    

P(triangle) = 0.500672 in 1000000 tries


## Sum of powers (Faulhaber's formula)


**Adapted from the [Wikipedia article on Bernoulli numbers](https://en.wikipedia.org/wiki/Bernoulli_number)**


Bernoulli numbers feature in the expression for the sum of the $m$th powers of the first $n$ positive integers. For $m, n \ge 0$ define


$$S_m(n) = \sum_{k=1}^n k^m = 1^m + 2^m + \cdots + n^m$$


The idea is that you want to sum some $m$th powers for numbers $1, 2, 3, \cdots$ up to $n$.
The conclusion is that this sum can always be written as $\frac{n^{m+1}}{m+1}$ plus some other 
smaller powers of $n$, each multiplied by constants. We can write this abstractly:


$$1^m+2^m+3^m+\cdots + n^m = \frac{n^{m+1}}{m+1} + \alpha_1 \cdot n + \alpha_2 \cdot n^2 + \cdots + \alpha_m \cdot n^m$$

where the challenge is to determine the values of $\alpha_i$. These will vary with $m$ of course.
Hence to add up $1 + 2 + 3 + \cdot + n$ you will square $n$ since $m=1$ and so $m+1=2$; *n-squared*. If we 
take a sum of cubes from 1 to 5 we can casually say


$S_3(5) = 1^3 + 2^3 + 3^3 + 4^3 + 5^3 = 225 = \frac{5^4}{4} + $ some smaller powers of $5$ times some constants.


In fact the formula for summing cubes has $alpha_1=0$, $\alpha_2=1/4$ and $alpha_3=1/2$ and so


$S_3(5)=5^4/4+5^3/2+5^2/4=225$ so that works out nicely.


The coefficients $\alpha_i$ in these polynomials are related to the **Bernoulli numbers**
according to ***Bernoulli's formula*** which is also known as ***Faulhaber's formula***


$$S_m(n) = \frac{1}{m + 1} \sum_{k=0}^m \binom{m + 1}{k} B^+_k n^{m + 1 - k}$$


where $\binom{m + 1}{k}$ is the binomial coefficient $(m+1)$ choose $k$.


Some people use a slightly different convention for Bernoulli numbers and give Bernoulli's formula in this way:


$$S_m(n) = \frac{1}{m + 1} \sum_{k=0}^m (-1)^k \binom{m + 1}{k} B^-_k n^{m + 1 - k}$$.


Now there remain only two things: Describing how to derive the Bernoulli numbers and listing some of them. 
Deriving them is not done here; for the moment we will just take them as *given*. Perhaps they fell out of 
the sky. Remember there are two versions of the Bernoulli numbers, the + and - versions. These differ only
by the sign of $B_1^\pm$:


$$B_0^+=1,\;\; B_1^+=\frac{1}{2},\;\; B_2^+=\frac{1}{6},\;\; B_3^+=0,\;\; B_4^+=\frac{-1}{30},\;\; B_5^+=0,\;\; B_6^+=\frac{1}{42},\;\; B_7^+=0,\;\; B_8^+=\frac{-1}{30},\;\; \cdots$$

$$B_0^-=1,\;\; B_1^-=\frac{-1}{2},\;\; B_2^-=\frac{1}{6},\;\; B_3^-=0,\;\; B_4^-=\frac{-1}{30},\;\; B_5^-=0,\;\; B_6^-=\frac{1}{42},\;\; B_7^-=0,\;\; B_8^-=\frac{-1}{30},\;\; \cdots$$


So we can notice that these are identical except for $B_1$ and that all of the odd ones seem to be $0$ for
$B_3$, $B_5$ and so on. (This pattern holds true: All the odd Bernoulli numbers from 3 up are zero.)


Now if we need to sum a bunch of numbers raised to the 7th power we can produce a specific formula. Let's use
the abstract formula given above to calculate all of the coefficients for $S_7(n)$. In fact let's do it twice
since we have two versions of the formula and I want to make sure they agree with one another.

First formula way: 


$$S_m(n) = \frac{1}{m + 1} \sum_{k=0}^m \binom{m + 1}{k} \;\cdot\; B^+_k \;\cdot\; n^{m + 1 - k}$$


$$S_7(n) = \frac{1}{8} \sum_{k=0}^7 \binom{8}{k} \;\cdot\; B^+_k \;\cdot\; n^{8 - k}$$


$$S_7(n)=\frac{1}{8}\cdot \left( 
\binom{8}{0} \cdot 1 \cdot n^{8} +
\binom{8}{1} \cdot \frac{1}{2} \cdot n^{7} +
\binom{8}{2} \cdot \frac{1}{6} \cdot n^{6} +
\binom{8}{3} \cdot 0 \cdot n^{5} +
\binom{8}{4} \cdot \frac{-1}{30} \cdot n^{4} +
\binom{8}{5} \cdot 0 \cdot n^{3} +
\binom{8}{6} \cdot \frac{1}{42} \cdot n^{2} +
\binom{8}{7} \cdot 0 \cdot n^{1}
\right)$$


$$S_7(n)=\frac{1}{8}\cdot \left( 
1 \cdot 1 \cdot n^{8} +
8 \cdot \frac{1}{2} \cdot n^{7} +
28 \cdot \frac{1}{6} \cdot n^{6} +
70 \cdot \frac{-1}{30} \cdot n^{4} +
28 \cdot \frac{1}{42} \cdot n^{2} +
\right)$$


$$S_7(n)= 
\frac{1}{8} \cdot n^{8} +
\frac{1}{2} \cdot n^{7} +
\frac{7}{12} \cdot n^{6} +
\frac{-7}{24} \cdot n^{4} +
\frac{2}{24} \cdot n^{2}$$



Second formula way: 


$$S_m(n) = \frac{1}{m + 1} \sum_{k=0}^m (-1)^k \binom{m + 1}{k} \;\cdot\; B^-_k \;\cdot\; n^{m + 1 - k}$$.


$$S_7(n) = \frac{1}{8} \sum_{k=0}^7 (-1)^k \binom{8}{k} \;\cdot\; B^-_k \;\cdot\; n^{8 - k}$$


$$S_7(n)=\frac{1}{8}\cdot \left( 
(-1)^0 \cdot \binom{8}{0} \cdot 1 \cdot n^{8} +
(-1)^1 \cdot \binom{8}{1} \cdot \frac{1}{2} \cdot n^{7} +
(-1)^2 \cdot \binom{8}{2} \cdot \frac{1}{6} \cdot n^{6} +
(-1)^3 \cdot \binom{8}{3} \cdot 0 \cdot n^{5} +
(-1)^4 \cdot \binom{8}{4} \cdot \frac{-1}{30} \cdot n^{4} +
(-1)^5 \cdot \binom{8}{5} \cdot 0 \cdot n^{3} +
(-1)^6 \cdot \binom{8}{6} \cdot \frac{1}{42} \cdot n^{2} +
(-1)^7 \cdot \binom{8}{7} \cdot 0 \cdot n^{1}
\right)$$


$$S_7(n)=\frac{1}{8}\cdot \left( 
1 \cdot 1 \cdot 1 \cdot n^{8} +
-1 \cdot 8 \cdot \frac{-1}{2} \cdot n^{7} +
1 \cdot 28 \cdot \frac{1}{6} \cdot n^{6} +
1 \cdot 70 \cdot \frac{-1}{30} \cdot n^{4} +
1 \cdot 28 \cdot \frac{1}{42} \cdot n^{2} +
\right)$$


$$S_7(n)= 
\frac{1}{8} \cdot n^{8} +
\frac{1}{2} \cdot n^{7} +
\frac{7}{12} \cdot n^{6} +
\frac{-7}{24} \cdot n^{4} +
\frac{2}{24} \cdot n^{2}$$


So both versions of the general formulas produce the same specific formula. 
Now we could write some Python code to check that these
formulas work for a number of $m$ and $n$ cases just to be sure. 











In [27]:
# This code compares a sum of powers to the Bernoulli formula for the same
#   There are two parameters 
#     n is the end of the sum: 1, 2, 3, 4, ..., n
#     m is the power that these numbers are raised to; then we sum
# 
# The two methods (summing directly and the Bernoulli formula) agree
#   very well but the Bernoulli formula is subject to rounding errors
#   for larger values of n. 
#

m_low = 1
m_hi = 5
n_low = 1
n_hi = 30

import numpy as np
from scipy.misc import comb

B = [1., -1./2., 1./6., 0., -1./30., 0., 1/42., 0., -1./30.]

def Bernoulli(m, n):
    sum = 0.
    for i in range(n):
        sum += np.power(i+1,m)
    formula = 0.
    for k in range(m+1):
        term1 = 1./(m+1.)
        term2 = np.power(-1., k)
        term3 = comb(m+1, k)
        term4 = B[k]
        term5 = np.power(n, m+1-k)
        formula += term1 * term2 * term3 * term4 * term5
    
    # un-comment the next line to see all the results
    # print m, n, sum, formula, round(sum-formula, 12)
    return np.abs(sum - formula)

nErrors = 0
errorSum = 0.
for m in range(m_low,m_hi+1):
    for n in range(n_low,n_hi+1):
        error = Bernoulli(m,n)
        if error != 0.: 
            nErrors += 1
            errorSum += error
            
print '\nTotal not quite equal = ', nErrors, 'with total absolute sum = ', errorSum, '\n\n'


Total not quite equal =  35 with total absolute sum =  2.19792836154e-08 


